## Data Scraping

#### In this notebook, I extract HCPCS codes (Type A) from an Excel file and utilize Selenium to check the existence of these codes on the Medicare website. Given the lack of an available database containing the corresponding outpatient prices, Selenium is employed to retrieve the necessary pricing information directly from the website.

In [14]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [15]:
Codes_and_descrption = pd.read_excel('full_filtered_hcpcs_results.xlsx')

In [16]:
Codes_and_description = Codes_and_descrption[~Codes_and_descrption['Info'].str.contains('Error', case=False, na=False)]
Codes_and_description['Code']

0       10005
1       10007
2       10009
3       10021
4       10030
        ...  
2264    46257
2265    46258
2266    46260
2267    46261
2268    46262
Name: Code, Length: 2269, dtype: int64

In [17]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument("--headless")  
chrome_service = Service(r'C:\Program Files\chromedriver-win64\chromedriver.exe')  

driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

# Base URL for Medicare procedure cost lookup
base_url = "https://www.medicare.gov/procedure-price-lookup/cost/"

results = []

for code in Codes_and_description['Code']:
    url = base_url + str(code)
    
    driver.get(url)
    
    # Extract the second price (Hospital Outpatient Departments)
    try:
        # Target the second occurrence of the price element
        price_elements = driver.find_elements(By.XPATH, "//p[contains(@class, 'chart-inner-label-cost')]")
        if len(price_elements) > 1:
            outpatient_price = price_elements[1].text
        else:
            outpatient_price = "N/A"
    except Exception as e:
        outpatient_price = "N/A"
        print(f"Error: {e}")
    
    print(f"Code: {code}, Hospital Outpatient Price: {outpatient_price}")
    
    results.append({'Code': code, 'Hospital Outpatient Price': outpatient_price})

driver.quit()

results_df = pd.DataFrame(results)

results_df.to_csv('hospital_outpatient_prices.csv', index=False)


Code: 10005, Hospital Outpatient Price: $148
Code: 10007, Hospital Outpatient Price: $151
Code: 10009, Hospital Outpatient Price: $154
Code: 10021, Hospital Outpatient Price: N/A
Code: 10030, Hospital Outpatient Price: $159
Code: 10060, Hospital Outpatient Price: $59
Code: 10061, Hospital Outpatient Price: $111
Code: 10080, Hospital Outpatient Price: $154
Code: 10081, Hospital Outpatient Price: $167
Code: 10120, Hospital Outpatient Price: $95
Code: 10121, Hospital Outpatient Price: $344
Code: 10140, Hospital Outpatient Price: N/A
Code: 10160, Hospital Outpatient Price: $94
Code: 10180, Hospital Outpatient Price: $576
Code: 11000, Hospital Outpatient Price: N/A
Code: 11010, Hospital Outpatient Price: $188
Code: 11011, Hospital Outpatient Price: $192
Code: 11012, Hospital Outpatient Price: $622
Code: 11042, Hospital Outpatient Price: $86
Code: 11043, Hospital Outpatient Price: $148
Code: 11044, Hospital Outpatient Price: $352
Code: 11102, Hospital Outpatient Price: $45
Code: 11104, Hospi

In [22]:
hospital_fees = pd.read_csv('hospital_outpatient_prices.csv')
procedures = pd.read_excel('full_filtered_hcpcs_results.xlsx')

In [24]:
Procedural_costs = pd.merge(procedures, hospital_fees, on='Code', how='inner')

In [26]:
Procedural_costs.to_csv('Procedural_costs.csv', index=False)